In [3]:
pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 51.0 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

/usr/local/lib/python3.10/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [6]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,2.698485e+10,1.0,1004906.0,1.0,1.39,364.0,-0.6,1631.0,1.0,0.0,0.0
1,2375,2.698485e+10,1.0,1033142.0,1.0,0.82,364.0,0.0,1631.0,1.0,0.0,0.0


In [7]:
data_train = data[data['week_no'] < data['week_no'].max() - 3]
data_test = data[data['week_no'] >= data['week_no'].max() - 3]

In [8]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[827656.0, 831447.0, 845896.0, 852662.0, 85694..."
1,2,"[854852.0, 930118.0, 1077555.0, 1098066.0, 556..."


In [10]:
result_table = pd.DataFrame({'name': [], 'precision@3': [], 'precision@5': [],
                             'recall@3': [], 'recall@5': []})

In [11]:
def precision_at_k(recommended_list, bought_list, k):
    flags = np.isin(np.array(recommended_list)[:k], np.array(bought_list))
    return flags.sum() / len(recommended_list)

def recall_at_k(recommended_list, bought_list, k):
    flags = np.isin(np.array(recommended_list)[:k], np.array(bought_list))
    return flags.sum() / len(bought_list)

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [9]:
def weighted_random_recommendation(iw, n):
    """Случайные рекоммендации с учетом весов всех item"""
    recs = np.random.choice(iw['item_id'], n, p=iw['item_weight'], replace=False)
    return recs.tolist()

# найти все веса
iw = data_train.groupby('item_id')['sales_value'].sum().reset_index()
iw.columns = ['item_id', 'sales_value_sum']
iw['item_weight'] = np.log(iw['sales_value_sum'] + 1)
iw['item_weight'] = iw['item_weight'] / iw['item_weight'].sum()
iw = iw[['item_id','item_weight']]

In [12]:
def get_weighted_random_recommendation(col_name, result_table=result_table):
    precision_list, recall_list = [], []
    for num in [3, 5]:
        result[col_name] =\
            result['user_id'].apply(lambda x: weighted_random_recommendation(iw, n=num))
        precision_list.append(
            result.apply(lambda row: precision_at_k(row[col_name], row['actual'], k=num), axis=1).mean())
        recall_list.append(
            result.apply(lambda row: recall_at_k(row[col_name], row['actual'], k=num), axis=1).mean())

    result_table.loc[len(result_table)] = [col_name, precision_list[0], precision_list[1],
                                           recall_list[0], recall_list[1]]


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [14]:
get_weighted_random_recommendation('weighted_random_recommendation')
display('рекомендации:', result.iloc[:, :2].join(result.iloc[:, -1:]).head(2), 'метрики:', result_table.tail(1))

'рекомендации:'

,user_id,actual,weighted_random_recommendation
0,1,"[827656.0, 831447.0, 845896.0, 852662.0, 85694...","[8293343.0, 1024032.0, 6632869.0, 851683.0, 84..."
1,2,"[854852.0, 930118.0, 1077555.0, 1098066.0, 556...","[1101445.0, 1086119.0, 5580525.0, 849578.0, 10..."


'метрики:'

,name,precision@3,precision@5,recall@3,recall@5
1,weighted_random_recommendation,0.001832,0.001538,0.000073,0.000187


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code